In [29]:
# necessary packages #

#using Pkg
#Pkg.add("Distances")
using Distributions
using Random
using Distances
using LinearAlgebra
using SparseArrays
using IterativeSolvers
using ProgressMeter
using JLD2

In [30]:
#include("../../../util2.j")

In [31]:
#using Pkg
#Pkg.add("ProgressMeter");

In [32]:
# Generate simulation data #

Random.seed!(123);
N = 1200 # sample size
Nobs = 1000;
q = 10; K = 50; 

sqrt_Σ_diag = sqrt.([0.5, 1, 0.4, 2, 0.3, 2.5, 3.5, 0.45, 1.5, 0.5]);

β = [1.0 -1.0 1.0 -0.5 2.0 -1.5 0.5 0.3 -2.0 1.5
     -5.0 2.0 3.0 -2.0 -6.0 4.0 5.0 -3.0 6.0 -4.0
     8.0 6.9 -12.0 0.0 -4.0 7.7 -8.8 3.3 6.6 -5.5];

ϕ = rand(Uniform(3 / sqrt(2), 30 / sqrt(2)), K);
Λ = rand(Uniform(-2, 2), K, q); # loading matrix

coords = rand(N, 2);                                          # random location over unit square
X = [fill(1, (N,)) rand(N) rand(N)];                             # design matrix
Dist = pairwise(Euclidean(), coords, dims = 1);                  # distance matrix
F_true = hcat([rand(MvNormal(exp.(-ϕ[i] * Dist)), 1) for i in 1:K]...);
ω = F_true * Λ;                                               # latent process
Y = X * β + ω + [rand(Normal(0.0, sqrt_Σ_diag[i]), 1)[1] for j in 1:N, i in 1:q];              # response matrix
ω_incp = ω + fill(1.0, (N, 1)) * transpose(β[1, :]);

In [33]:
# Some data preparations #

#ordx = sortperm(coords[:, 1]);                                # sort order based on the first coordinates
ordx = 1:N;
X_ord = X[ordx, :]; Y_ord = Y[ordx, :]; ω_ord = ω[ordx, :];   # sorted data
ω_incp_obs = ω_ord + fill(1.0, (N, 1)) * transpose(β[1, :]); # latent process + intercept
coords_ord = coords[ordx, :];
S_ind = [sample(1:N, Nobs, replace = false, ordered = true) for i in 1:q] 
                                                              # index of observed response for all q responses
S = sort(union([S_ind[i] for i in 1:q]...));                  # location with at least one observation
M_ind = [setdiff(S, S_ind[i]) for i in 1:q];                  # in S not in S1
obs_ind = vcat([S_ind[i] .+ (i - 1) * N for i in 1:q]...)     # index of the observed location for all response among N locations
perm_ind = sortperm(vcat([S_ind[i] for i in 1:q]...))

index_S = zeros(N);                                           # build index indicating which response are observed
index_S_M = fill(0.0, N, q);
for i in 1:q
    index_S_M[S_ind[i], i] .= 1;
    index_S = index_S + 2^(i - 1) * index_S_M[:, i]
end
M_Sind = [findall(x -> x == 0.0, index_S_M[S, i]) for i in 1:q];
                                                              # index of M1 among S 

In [34]:
@save "sim3data.jld"

In [35]:
# check the plot of the data 
using RCall
@rput ω_incp
@rput coords
R"""
library(MBA)
library(classInt)
library(RColorBrewer)
library(sp)
library(coda)
library(spBayes)
library(fields)
library(corrgram)

h <- 12
surf.w_incp1 <- mba.surf(cbind(coords, ω_incp[, 1]), no.X = 300, no.Y = 300, 
              exten = TRUE, sp = TRUE, h = h)$xyz.est
surf.w_incp2 <- mba.surf(cbind(coords, ω_incp[, 2]), no.X = 300, no.Y = 300, 
              exten = TRUE, sp = TRUE, h = h)$xyz.est
surf.w_incp3 <- mba.surf(cbind(coords, ω_incp[, 3]), no.X = 300, no.Y = 300, 
              exten = TRUE, sp = TRUE, h = h)$xyz.est
surf.w_incp4 <- mba.surf(cbind(coords, ω_incp[, 4]), no.X = 300, no.Y = 300, 
              exten = TRUE, sp = TRUE, h = h)$xyz.est
surf.w_incp5 <- mba.surf(cbind(coords, ω_incp[, 5]), no.X = 300, no.Y = 300, 
              exten = TRUE, sp = TRUE, h = h)$xyz.est
surf.w_incp6 <- mba.surf(cbind(coords, ω_incp[, 6]), no.X = 300, no.Y = 300, 
              exten = TRUE, sp = TRUE, h = h)$xyz.est
surf.w_incp7 <- mba.surf(cbind(coords, ω_incp[, 7]), no.X = 300, no.Y = 300, 
              exten = TRUE, sp = TRUE, h = h)$xyz.est
surf.w_incp8 <- mba.surf(cbind(coords, ω_incp[, 8]), no.X = 300, no.Y = 300, 
              exten = TRUE, sp = TRUE, h = h)$xyz.est
surf.w_incp9 <- mba.surf(cbind(coords, ω_incp[, 9]), no.X = 300, no.Y = 300, 
              exten = TRUE, sp = TRUE, h = h)$xyz.est
surf.w_incp10 <- mba.surf(cbind(coords, ω_incp[, 10]), no.X = 300, no.Y = 300, 
              exten = TRUE, sp = TRUE, h = h)$xyz.est

surf.brks <- classIntervals(c(surf.w_incp1[["z"]], surf.w_incp2[["z"]], surf.w_incp3[["z"]], surf.w_incp4[["z"]],
                surf.w_incp5[["z"]], surf.w_incp6[["z"]], surf.w_incp7[["z"]], surf.w_incp8[["z"]],
                surf.w_incp9[["z"]], surf.w_incp10[["z"]]),500, 'pretty')$brks
col.pal <- colorRampPalette(brewer.pal(11,'RdBu')[11:1])
xlim <- c(0, 1.13)

zlim <- range(c(surf.w_incp1[["z"]], surf.w_incp2[["z"]], surf.w_incp3[["z"]], surf.w_incp4[["z"]],
                surf.w_incp5[["z"]], surf.w_incp6[["z"]], surf.w_incp7[["z"]], surf.w_incp8[["z"]],
                surf.w_incp9[["z"]], surf.w_incp10[["z"]]))

# size for the mapping of w               
width <- 720
height <- 720
pointsize <- 16

png(paste("../../pics/sim3_map-w1_incp-true.png", sep = ""), 
    width = width, height = height, pointsize = pointsize, family = "Courier")
par(mfrow = c(1, 1))
##Obs
i <- as.image.SpatialGridDataFrame(surf.w_incp1)
plot(coords, typ="n", cex=0.5, xlim=xlim, axes=FALSE, ylab="y", xlab="x") 
     #main = "true")
axis(2, las=1)
axis(1)
image.plot(i, add=TRUE, col=rev(col.pal(length(surf.brks)-1)), zlim=zlim)
dev.off()

png(paste("../../pics/sim3_map-w2_incp-true.png", sep = ""), 
    width = width, height = height, pointsize = pointsize, family = "Courier")
par(mfrow = c(1, 1))
##Obs
i <- as.image.SpatialGridDataFrame(surf.w_incp2)
plot(coords, typ="n", cex=0.5, xlim=xlim, axes=FALSE, ylab="y", xlab="x") 
     #main = "true")
axis(2, las=1)
axis(1)
image.plot(i, add=TRUE, col=rev(col.pal(length(surf.brks)-1)), zlim=zlim)
dev.off()

png(paste("../../pics/sim3_map-w3_incp-true.png", sep = ""), 
    width = width, height = height, pointsize = pointsize, family = "Courier")
par(mfrow = c(1, 1))
##Obs
i <- as.image.SpatialGridDataFrame(surf.w_incp3)
plot(coords, typ="n", cex=0.5, xlim=xlim, axes=FALSE, ylab="y", xlab="x") 
     #main = "true")
axis(2, las=1)
axis(1)
image.plot(i, add=TRUE, col=rev(col.pal(length(surf.brks)-1)), zlim=zlim)
dev.off()

png(paste("../../pics/sim3_map-w4_incp-true.png", sep = ""), 
    width = width, height = height, pointsize = pointsize, family = "Courier")
par(mfrow = c(1, 1))
##Obs
i <- as.image.SpatialGridDataFrame(surf.w_incp4)
plot(coords, typ="n", cex=0.5, xlim=xlim, axes=FALSE, ylab="y", xlab="x") 
     #main = "true")
axis(2, las=1)
axis(1)
image.plot(i, add=TRUE, col=rev(col.pal(length(surf.brks)-1)), zlim=zlim)
dev.off()

png(paste("../../pics/sim3_map-w5_incp-true.png", sep = ""), 
    width = width, height = height, pointsize = pointsize, family = "Courier")
par(mfrow = c(1, 1))
##Obs
i <- as.image.SpatialGridDataFrame(surf.w_incp5)
plot(coords, typ="n", cex=0.5, xlim=xlim, axes=FALSE, ylab="y", xlab="x") 
     #main = "true")
axis(2, las=1)
axis(1)
image.plot(i, add=TRUE, col=rev(col.pal(length(surf.brks)-1)), zlim=zlim)
dev.off()

png(paste("../../pics/sim3_map-w6_incp-true.png", sep = ""), 
    width = width, height = height, pointsize = pointsize, family = "Courier")
par(mfrow = c(1, 1))
##Obs
i <- as.image.SpatialGridDataFrame(surf.w_incp6)
plot(coords, typ="n", cex=0.5, xlim=xlim, axes=FALSE, ylab="y", xlab="x") 
     #main = "true")
axis(2, las=1)
axis(1)
image.plot(i, add=TRUE, col=rev(col.pal(length(surf.brks)-1)), zlim=zlim)
dev.off()

png(paste("../../pics/sim3_map-w7_incp-true.png", sep = ""), 
    width = width, height = height, pointsize = pointsize, family = "Courier")
par(mfrow = c(1, 1))
##Obs
i <- as.image.SpatialGridDataFrame(surf.w_incp7)
plot(coords, typ="n", cex=0.5, xlim=xlim, axes=FALSE, ylab="y", xlab="x") 
     #main = "true")
axis(2, las=1)
axis(1)
image.plot(i, add=TRUE, col=rev(col.pal(length(surf.brks)-1)), zlim=zlim)
dev.off()

png(paste("../../pics/sim3_map-w8_incp-true.png", sep = ""), 
    width = width, height = height, pointsize = pointsize, family = "Courier")
par(mfrow = c(1, 1))
##Obs
i <- as.image.SpatialGridDataFrame(surf.w_incp8)
plot(coords, typ="n", cex=0.5, xlim=xlim, axes=FALSE, ylab="y", xlab="x") 
     #main = "true")
axis(2, las=1)
axis(1)
image.plot(i, add=TRUE, col=rev(col.pal(length(surf.brks)-1)), zlim=zlim)
dev.off()

png(paste("../../pics/sim3_map-w9_incp-true.png", sep = ""), 
    width = width, height = height, pointsize = pointsize, family = "Courier")
par(mfrow = c(1, 1))
##Obs
i <- as.image.SpatialGridDataFrame(surf.w_incp9)
plot(coords, typ="n", cex=0.5, xlim=xlim, axes=FALSE, ylab="y", xlab="x") 
     #main = "true")
axis(2, las=1)
axis(1)
image.plot(i, add=TRUE, col=rev(col.pal(length(surf.brks)-1)), zlim=zlim)
dev.off()

png(paste("../../pics/sim3_map-w10_incp-true.png", sep = ""), 
    width = width, height = height, pointsize = pointsize, family = "Courier")
par(mfrow = c(1, 1))
##Obs
i <- as.image.SpatialGridDataFrame(surf.w_incp10)
plot(coords, typ="n", cex=0.5, xlim=xlim, axes=FALSE, ylab="y", xlab="x") 
     #main = "true")
axis(2, las=1)
axis(1)
image.plot(i, add=TRUE, col=rev(col.pal(length(surf.brks)-1)), zlim=zlim)
dev.off()


# correlation maps #
corr_resp = cor(ω_incp, use = "pairwise.complete.obs");
par(mfrow = c(1,1))
colnames(corr_resp) <- c(1:dim(ω_incp)[2])
rownames(corr_resp) <- c(1:dim(ω_incp)[2])
png(paste("../../pics/sim3_factor_corr_plot_raw.png", sep = ""), 
    width = width, height = height, pointsize = pointsize, family = "Courier")
par(mfrow = c(1, 1))
corrgram(corr_resp, order=FALSE, lower.panel=panel.shade, gap = 0.2,
         upper.panel=panel.pie, text.panel=panel.txt, main=" ",
         col.regions = colorRampPalette(c( "darkseagreen3",
                                           "white", "cadetblue3")))
dev.off()


"""

RObject{IntSxp}
png 
  2 
